**SLEEP CALL**

### 12S18027 Christopher A Hutabarat
### 12S18028 Alda G M Lumban Gaol
### 12S18032 Sarah H M Siahaan 

# **Analysis Emotion Of Comment Topic Reddit With Multi-label Classification Using SVM Algorithm**


In [22]:
import numpy as np
import pandas as pd
import re
import nltk
import string
from pandas import DataFrame as df
pd.options.mode.chained_assignment = None

In [23]:
df_go1 = pd.read_csv("goemotions_1.csv")
df_go2 = pd.read_csv("goemotions_2.csv")
df_go3 = pd.read_csv("goemotions_3.csv")

df_go_emotion = pd.concat([df_go1, df_go2, df_go3])

df_go_emotion.describe()

,created_utc,rater_id,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
count,2.112250e+05,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,...,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000,211225.000000
mean,1.547659e+09,37.619742,0.081103,0.043768,0.038272,0.064472,0.083418,0.028401,0.034840,0.045885,...,0.038779,0.008569,0.041259,0.006164,0.041591,0.006102,0.011954,0.031994,0.026105,0.261797
std,7.696842e+05,23.070528,0.272994,0.204580,0.191853,0.245592,0.276514,0.166116,0.183374,0.209236,...,0.193067,0.092172,0.198890,0.078269,0.199653,0.077880,0.108679,0.175985,0.159448,0.439614
min,1.546301e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.546993e+09,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.547664e+09,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.548320e+09,57.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.548979e+09,81.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# **Data Cleaning**

### **1. Drop kolom yang tidak dibutuhkan**

In [24]:
to_drop = ['author', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear']
df_go_emotion.drop(columns=to_drop, inplace=True)

df.head(df_go_emotion)

,text,id,subreddit,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,nrl,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,unpopularopinion,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,confessions,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,facepalm,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,starwarsspeculation,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### **2. Cek data yang kosong (Missing Value)**

In [25]:
df_go_emotion.isna()

,text,id,subreddit,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71220,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71221,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71222,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71223,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### **3. Cek keunikan id sebagai index pada dataset**



In [26]:
df_go_emotion['id'].is_unique

False

### **4. Menghapus nilai yang sama pada dataset**



In [27]:
df_go_emotion.drop_duplicates(subset='id').reset_index(drop=True)

,text,id,subreddit,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,nrl,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,unpopularopinion,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,confessions,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,facepalm,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,starwarsspeculation,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58006,He called [NAME] to the Steelers and was outsp...,eed7qdq,steelers,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
58007,I didn’t even notice that one. I noticed the b...,ef0lnq5,AnimalsBeingBros,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
58008,"Thanks, [NAME]",edirq0m,unpopularopinion,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58009,Everyone likes [NAME].,ee6pagw,heroesofthestorm,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


# **Text Preprocessing**

### **1. Case Folding**

In [28]:
df_go_emotion["text"] = df_go_emotion["text"].str.lower()

df_go_emotion.head()

,text,id,subreddit,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,that game hurt.,eew5j0j,nrl,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category i...,eemcysk,unpopularopinion,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"you do right, if you don't care then fuck 'em!",ed2mah1,confessions,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,man i love reddit.,eeibobj,facepalm,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[name] was nowhere near them, he was by the fa...",eda6yn6,starwarsspeculation,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### **2. Removal of Punctuations**

In [29]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df_go_emotion["text"] = df_go_emotion["text"].apply(lambda text: remove_punctuation(text))

df_go_emotion.head()

,text,id,subreddit,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,that game hurt,eew5j0j,nrl,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,sexuality shouldn’t be a grouping category it...,eemcysk,unpopularopinion,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,you do right if you dont care then fuck em,ed2mah1,confessions,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,man i love reddit,eeibobj,facepalm,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,name was nowhere near them he was by the falcon,eda6yn6,starwarsspeculation,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
